# SKU price prediction

In [55]:
# Data manipulation
import datetime
import numpy as np
import pandas as pd

# Visualizations
import matplotlib.pyplot as plt
import seaborn as sns


### 1. Loading Data

In [56]:
sku_first = pd.read_csv('data/first_group_sku.csv')
sku_second = pd.read_csv('data/second_group_sku.csv')
sku_third= pd.read_csv('data/third_group_sku.csv')

### 2. Concat data: merged_data 

In [57]:
concated_skus = pd.concat([sku_first, sku_second, sku_third])
concated_skus.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6775 entries, 0 to 2780
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   dates          6775 non-null   object 
 1   SKU            6775 non-null   int64  
 2   price_per_sku  6699 non-null   float64
 3   num_purchases  6775 non-null   int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 264.6+ KB


In [58]:
concated_skus = concated_skus.dropna(subset=['price_per_sku'])
concated_skus.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6699 entries, 0 to 2780
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   dates          6699 non-null   object 
 1   SKU            6699 non-null   int64  
 2   price_per_sku  6699 non-null   float64
 3   num_purchases  6699 non-null   int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 261.7+ KB


In [59]:
# Объедините данные по SKU, year, week_num и дате
concated_skus['dates'] = pd.to_datetime(concated_skus['dates'])
concated_skus['month'] = concated_skus['dates'].dt.month
concated_skus['day'] = concated_skus['dates'].dt.day
concated_skus.drop(columns = ['dates', 'num_purchases'], axis = 1, inplace = True)
# concated_skus['day_name'] = concated_skus['dates'].dt.day_name()

# # merged_concated_skus['dates']
# # merged_conc_data = pd.merge(merged_concated_skus, conc_df, left_on=["SKU", merged_concated_skus['dates'].dt.year, merged_concated_skus['dates'].dt.isocalendar().week], right_on=["SKU", "year", "week_num"], how="left")
# concated_skus = pd.merge(concated_skus, promo_df, left_on=["SKU", concated_skus['dates'].dt.year, concated_skus['dates'].dt.isocalendar().week], right_on=["SKU", "year", "week_num"], how="left")
# # concated_skus.drop(сolumns=['dates'], axis=1, inplace=True)
# concated_skus['discount'].fillna(1, inplace=True)
# concated_skus.drop(columns = ['dates', 'num_purchases', 'group', 'discount'], axis = 1, inplace = True)

# concated_skus.info()


## Modeling

In [60]:
from catboost import CatBoostRegressor
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV


random_state = 42
train = concated_skus.copy()

X_train = train.drop(['price_per_sku'], axis=1)
Y_train = train['price_per_sku']
# columns = X_train.columns

# X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
# X_train.head()

In [61]:
%%time
model = CatBoostRegressor(learning_rate = 0.1, loss_function='RMSE', random_state=random_state)
model.fit(X_train, Y_train)

0:	learn: 50.2851085	total: 1.03ms	remaining: 1.03s
1:	learn: 47.3107134	total: 1.73ms	remaining: 864ms
2:	learn: 44.3128755	total: 2.46ms	remaining: 817ms
3:	learn: 41.4090180	total: 3.4ms	remaining: 848ms
4:	learn: 38.8917606	total: 4.09ms	remaining: 815ms
5:	learn: 36.7762211	total: 4.89ms	remaining: 811ms
6:	learn: 35.1469703	total: 5.57ms	remaining: 790ms
7:	learn: 33.8193778	total: 6.34ms	remaining: 786ms
8:	learn: 32.4458507	total: 7.18ms	remaining: 790ms
9:	learn: 31.3271590	total: 8ms	remaining: 792ms
10:	learn: 29.8132538	total: 8.78ms	remaining: 790ms
11:	learn: 28.7750298	total: 9.52ms	remaining: 784ms
12:	learn: 27.3493160	total: 10.2ms	remaining: 778ms
13:	learn: 26.3833809	total: 10.9ms	remaining: 767ms
14:	learn: 25.2603175	total: 11.7ms	remaining: 765ms
15:	learn: 24.2269366	total: 12.4ms	remaining: 762ms
16:	learn: 23.4687049	total: 13.1ms	remaining: 759ms
17:	learn: 22.5686732	total: 14.1ms	remaining: 767ms
18:	learn: 21.7870139	total: 15.2ms	remaining: 787ms
19:	lea

In [62]:
X_train.head()

,SKU,month,day
0,32950,4,20
1,32950,4,24
2,32950,4,27
4,32950,5,3
5,32950,5,4


In [63]:
test_data = pd.read_csv('data/test.csv')
test_data

test_data['dates'] = pd.to_datetime(test_data['dates'])
test_data['month'] = test_data['dates'].dt.month
test_data['day'] = test_data['dates'].dt.day
test_data['year'] = test_data['dates'].dt.year
test_data['week_num'] = test_data['dates'].dt.isocalendar().week


# test_data['day_name'] = test_data['dates'].dt.day_name()
selected_columns = ['dates', 'SKU']
prediction = test_data[selected_columns]

test_data.drop(columns = ['dates'], axis = 1, inplace = True)
test_data.head()


,SKU,month,day,year,week_num
0,98255,12,17,2018,51
1,98255,12,21,2018,51
2,98255,12,22,2018,51
3,98255,12,23,2018,51
4,98255,12,24,2018,52


In [64]:

X_test = test_data
# X_test['price_per_sku'] = model.predict(X_test).round()

prediction['price_per_sku'] = model.predict(X_test).round()

prediction.to_csv('data/prediction.csv', index=False)
prediction

/var/folders/9t/d2tzjb412n723my412tvgtth0000gn/T/ipykernel_45915/1713157133.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prediction['price_per_sku'] = model.predict(X_test).round()


,dates,SKU,price_per_sku
0,2018-12-17,98255,67.0
1,2018-12-21,98255,66.0
2,2018-12-22,98255,66.0
3,2018-12-23,98255,66.0
4,2018-12-24,98255,66.0
...,...,...,...
170,2019-12-31,80631,84.0
171,2019-12-31,19632,152.0
172,2019-12-31,84395,131.0
173,2019-12-31,43678,139.0
